5.

In [101]:
class Dict():
    def __init__(self, capacity):
        self.capacity = capacity
        self.hash_table = [(None,None) for i in range(self.capacity)]
        
    def hash1(self,x):
        return x % self.capacity
        
    def put(self, key, value):
        hash_k = self.hash1(key)
        
        while self.hash_table[hash_k][0] != None:
            if self.hash_table[hash_k][0] == key:
                self.hash_table[hash_k] = (key,value)
                return
            else:
                hash_k = (hash_k + 1) % self.capacity
        self.hash_table[hash_k] = (key,value)
        
    def get(self, key):
        hash_k = self.hash1(key)
        h_key, value = self.hash_table[hash_k]
        if h_key == key:
            return value
        else:
            for i in range(self.capacity):
                hash_k = self.hash1(key+i)
                h_key, value = self.hash_table[hash_k]
                if h_key == key:
                    return value
        return None
    
class OuterNode():
    def __init__(self, x, capacity):
        self.key = x
        self.table = Dict(capacity)
        self.next = None
        
    def add(self,outernode):
        self.next = outernode
        
    def get(self, y):
        return self.table.get(y)
        
    def put(self, y, value):
        self.table.put(y, value)
            
class OuterTable:
    def __init__(self, size_x, size_y):
        self.M = size_x // 3 + 1
        self.size_x = size_x
        self.size_y = size_y
        self.ht = [None] * self.M

    def hash1(self, key):
        return key % self.M
    
    def get(self, key1, key2):
        i = self.hash1(key1)
        x = self.ht[i]
        while (x!=None):
            if (x.key==key1):
                return x.get(key2)
            else:
                x = x.next
        return None
    
    def put(self, key1, key2, value):
        i = self.hash1(key1)
        x = self.ht[i]
        while (x!=None):
            if (x.key==key1): 
                x.put(key2, value)
                return
            else:
                x = x.next
        n = OuterNode(key1, self.size_y * 2)
        n.put(key2, value)
        n.add(self.ht[i])
        self.ht[i] = n 
        
def generate(matrix):
    size_x = len(matrix)
    size_y = len(matrix[0])
    table = OuterTable(size_x, size_y)
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] != 0:
                table.put(i, j, matrix[i][j])
    return table

def getAllEntries(table):
    result = []
    for i in table.ht:
        while i != None:
            for j in i.table.hash_table:
                if j[0] != None:
                    result.append((i.key, j[0], j[1]))
            i = i.next
    return result

def addition(table1, table2):
    if not (table1.size_x == table2.size_x and table1.size_y == table2.size_y):
        return None
    for i in getAllEntries(table1):
        current = table2.get(i[0],i[1])
        if current != None:
            table2.put(i[0], i[1], i[2] + current)
        else:
            table2.put(i[0], i[1], i[2])
    return table2

def toMatrix(table):
    if table == None:
        return None
    matrix = [[0 for j in range(table.size_y)] for k in range(table.size_x)]
    for i in getAllEntries(table):
        matrix[i[0]][i[1]] = i[2]
    return matrix

def multiplication(table1, table2):
    if not (table1.size_y == table2.size_x):
        return None
    table3 = OuterTable(table1.size_x, table2.size_y)
    for i in getAllEntries(table1):
         for j in getAllEntries(table2):
            if i[1] == j[0]:
                current = table3.get(i[0], j[1])
                if current != None:
                    table3.put(i[0], j[1], current + i[2] * j[2])
                else:
                    table3.put(i[0], j[1], i[2] * j[2])
    return table3

In [102]:
def normalAdd(m1, m2):
    if not (len(m1) == len(m2) and len(m1[0]) == len(m2[0])):
        return None
    result = [[0 for j in range(len(m1[0]))] for k in range(len(m1))]
    for i in range(len(m1)):
        for j in range(len(m1[0])):
            result[i][j] = m1[i][j] + m2[i][j]
    return result
    
def normalMul(m1, m2):
    if not (len(m1[0]) == len(m2)):
        return None
    result = [[0 for j in range(len(m2[0]))] for k in range(len(m1))]
    for i in range(len(m1)):
        for j in range(len(m1[0])):
            for k in range(len(m2[0])):
                result[i][k] += m1[i][j] * m2[j][k]
    return result

In [103]:
matrix = [[0,0,0,0,0,0,0,2,0,0],
          [0,0,0,0,0,0,7,0,0,0],
          [0,0,0,0,0,0,0,0,0,0],
          [0,4,0,0,3,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,9],
          [1,0,0,0,0,0,0,0,0,0]
         ]
table = generate(matrix)

matrix2 = [[0,0,0],
           [8,0,0],
           [0,0,1],
           [0,0,0],
           [0,5,0],
           [0,0,0],
           [7,0,0],
           [0,6,0],
           [0,0,0],
           [0,0,9]
         ]
table2 = generate(matrix2)

matrix3 = [[0,4,0,0,0,0,0,0,6,0],
          [0,0,0,1,0,0,0,0,0,0],
          [0,3,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,7,0,0,0,0],
          [0,0,0,5,0,0,0,0,0,0]]
table3 = generate(matrix3)

In [104]:
import timeit
s1 = timeit.default_timer()

toMatrix(addition(table, table3))

e1 = timeit.default_timer()
t1 = e1 - s1

s2 = timeit.default_timer()

normalAdd(matrix, matrix3)

e2 = timeit.default_timer()
t2 = e2 - s2

print(t1)
print(t2)
t1 > t2

8.080000020527223e-05
3.2899999951041536e-05


True

In [105]:
s1 = timeit.default_timer()

toMatrix(multiplication(table, table2))

e1 = timeit.default_timer()
t1 = e1 - s1

s2 = timeit.default_timer()

normalMul(matrix, matrix2)

e2 = timeit.default_timer()
t2 = e2 - s2

print(t1)
print(t2)
t1 > t2

8.58000000789616e-05
5.020000003241876e-05


True

In [106]:
import sys
print(sys.getsizeof(table))
print(sys.getsizeof(matrix))

56
112


In [88]:
import numpy as np
np.dot(matrix, matrix2)

array([[ 0, 12,  0],
       [49,  0,  0],
       [ 0,  0,  0],
       [32, 15,  0],
       [ 0,  0, 81],
       [ 0,  0,  0]])

In [90]:
np.add(matrix, matrix3)

array([[0, 4, 0, 0, 0, 0, 0, 2, 6, 0],
       [0, 0, 0, 1, 0, 0, 7, 0, 0, 0],
       [0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 4, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 7, 0, 0, 0, 9],
       [1, 0, 0, 5, 0, 0, 0, 0, 0, 0]])